# Quantum. Project 3

Author:
- ***Nikita Makarevich (Student ID: 153989)***

## Imports and Utilities

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer
import qiskit.visualization as qvis
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
import numpy as np
from functools import partial

# selection of quantum simulator (or processor)
backend = Aer.get_backend("qasm_simulator")

In [2]:
def run_experiments(
    backend, circuit: QuantumCircuit, *, shots: int = 2048, runs: int = 3
) -> list[dict]:
    compiled_circuit = transpile(circuit, backend)
    return [
        backend.run(compiled_circuit, shots=shots).result().get_counts()
        for _ in range(runs)
    ]

In [3]:
@dataclass
class Setup:
    n: int
    qx: QuantumRegister
    cx: ClassicalRegister
    circuit: QuantumCircuit

    @classmethod
    def create(cls) -> "Setup":
        n = 2
        qx = QuantumRegister(n, "q")
        cx = ClassicalRegister(n, "c")
        circ = QuantumCircuit(qx, cx)
        return cls(n, qx, cx, circ)

## Circuit Elements

In [4]:
def connect_bell_state(setup: Setup) -> None:
    setup.circuit.x(setup.qx[0])
    setup.circuit.x(setup.qx[1])
    setup.circuit.h(setup.qx[0])
    setup.circuit.cx(setup.qx[0], setup.qx[1])


def connect_barrier(setup: Setup) -> None:
    qbits = [setup.qx[i] for i in range(setup.n)]
    setup.circuit.barrier(*qbits)


def connect_x_hat(setup: Setup, qbit: int) -> None:
    setup.circuit.h(setup.qx[qbit])


def connect_w_hat(setup: Setup, qbit: int) -> None:
    setup.circuit.s(setup.qx[qbit])
    setup.circuit.h(setup.qx[qbit])
    setup.circuit.t(setup.qx[qbit])
    setup.circuit.h(setup.qx[qbit])


def connect_z_hat(setup: Setup, qbit: int) -> None:
    return


def connect_v_hat(setup: Setup, qbit: int) -> None:
    setup.circuit.s(setup.qx[qbit])
    setup.circuit.h(setup.qx[qbit])
    setup.circuit.tdg(setup.qx[qbit])
    setup.circuit.h(setup.qx[qbit])


def connect_measurements(setup: Setup) -> None:
    for i in range(setup.n):
        setup.circuit.measure(setup.qx[i], setup.cx[i])

## E91 Protocol

In [5]:
def _key_to_string(key: np.ndarray) -> str:
    return "".join(map(str, (0.5 - 0.5 * key).astype(int)))


def run_e91_protocol(n_singlets: int):
    alice_mapping = {
        1: partial(connect_x_hat, qbit=0),
        2: partial(connect_w_hat, qbit=0),
        3: partial(connect_z_hat, qbit=0),
    }
    bob_mapping = {
        1: partial(connect_w_hat, qbit=1),
        2: partial(connect_z_hat, qbit=1),
        3: partial(connect_v_hat, qbit=1),
    }

    b = np.random.randint(1, 4, size=n_singlets)
    b_prime = np.random.randint(1, 4, size=n_singlets)

    a = []
    a_prime = []

    for b_i, b_i_prime in zip(b, b_prime):
        setup = Setup.create()
        connect_bell_state(setup)

        # Alice's choice
        alice_mapping[b_i](setup)

        # Bob's choice
        bob_mapping[b_i_prime](setup)

        connect_measurements(setup)

        result = run_experiments(backend, setup.circuit, shots=1, runs=1)[0]
        bitstring = list(result.keys())[0]
        a.append(int(bitstring[-1]))  # Alice's bit
        a_prime.append(int(bitstring[-2]))  # Bob's bit

    a = np.array(a)
    a_prime = np.array(a_prime)

    a = 1 - 2 * a  # Convert to +1/-1
    a_prime = 1 - 2 * a_prime  # Convert to +1/-

    # Key extraction

    key_mask = ((b == 2) & (b_prime == 1)) | ((b == 3) & (b_prime == 2))
    k = a[key_mask]
    k_prime = -1 * a_prime[key_mask]

    print(f"Alice key:\n\t{_key_to_string(k)}")
    print(f"Bob key:\n\t{_key_to_string(k_prime)}")

    same_length = len(k) == len(k_prime)
    print(
        f"Keys lengths: {len(k)} (Alice), {len(k_prime)} (Bob) - Same length: {same_length}"
    )
    if same_length:
        print("Num of mismatching bits:", np.sum(k != k_prime))

    # CHSH correlation calculation

    print("\n" + "=" * 40 + "\n")
    chsh_pairs = [
        (1, 1),
        (1, 3),
        (3, 1),
        (3, 3),
    ]
    correlations = []
    for b_val, b_prime_val in chsh_pairs:
        pair_mask = (b == b_val) & (b_prime == b_prime_val)
        a_pair = a[pair_mask]
        a_prime_pair = a_prime[pair_mask]
        correlation = np.mean(a_pair * a_prime_pair)
        correlations.append(correlation)
        print(f"Correlation for (b, b') = ({b_val}, {b_prime_val}): {correlation:.3f}")

    S = np.dot([1, -1, 1, 1], correlations)
    print(f"CHSH Correlation value S: {S:.3f}")
    print(f"Reference value of -2√2: {-2 * np.sqrt(2):.3f}")

In [7]:
run_e91_protocol(n_singlets=1024)

Alice key:
	111101001011110011100110101101010011011001000111111010001110101100111001010101000010111010111000101010101111011001011100101101010110100110010010000011001000111110000001101000001111110110101011001101000010001000000111110001110000011
Bob key:
	111101001011110011100110101101010011011001000111111010001110101100111001010101000010111010111000101010101111011001011100101101010110100110010010000011001000111110000001101000001111110110101011001101000010001000000111110001110000011
Keys lengths: 231 (Alice), 231 (Bob) - Same length: True
Num of mismatching bits: 0


Correlation for (b, b') = (1, 1): -0.735
Correlation for (b, b') = (1, 3): 0.643
Correlation for (b, b') = (3, 1): -0.700
Correlation for (b, b') = (3, 3): -0.835
CHSH Correlation value S: -2.912
Reference value of -2√2: -2.828
